Topic modeling with Top2Vec

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0', ignore_verifications=True)

In [ ]:
from datasets import get_dataset_split_names
get_dataset_split_names("cnn_dailymail", '3.0.0')

In [ ]:
import pandas as pd

import re

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

import spacy
nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")

import random

In [ ]:
dataset = load_dataset("cnn_dailymail", '3.0.0', split="train")


In [ ]:
dataframe = pd.DataFrame(dataset)
df = dataframe.loc[0:9999]

In [ ]:
df.to_csv(r'C:\Users\kayir\Documents\project_data\my_dataframe.csv')

In [ ]:
articles = df["article"].tolist()

In [ ]:
model = Top2Vec(articles)

In [ ]:
model.save("mymodel")

In [ ]:
topic_sizes, topic_nums = model.get_topic_sizes()
print(topic_sizes)
#список всех тем - тема с индексом 0 самая популярная и по ней есть 313 документов

In [ ]:
print(topic_nums)
#в модели 115 разныx тем

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(10)

In [ ]:
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f"Words: {words}")

In [ ]:
#ПОИСК ТЕМ ПО КЛЮЧЕВОМУ СЛОВУ
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["music"], num_topics=5)
print(topic_nums)
print(topic_scores)

In [ ]:
#ПОСМОТРЕТЬ ДОКУМЕНТЫ ПО ТЕМЕ, ЗНАЯ ЕЕ НОМЕР
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num = 19, num_docs = 5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document:{doc_id}, Score: {score}")
    print("----------")
    print(doc)
    print("----------")
    print()

In [ ]:
#ЗАПИСАТЬ РЕЗУЛЬТАТЫ В ДАТАФРЕЙМ
topic_of_interest = 5
topic_sizes, topic_nums = model.get_topic_sizes()
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=topic_of_interest, num_docs=topic_sizes[topic_of_interest])

results = list(zip(documents, document_scores, document_ids))
results_df = pd.DataFrame(results, columns = ['documents', 'document_scores', 'document_ids'])
results_df = results_df.drop_duplicates(subset="documents", keep='first')
results_df

In [ ]:
results_df.to_csv(r'C:\Users\kayir\Documents\project_data\politics.csv')

In [ ]:
#ОБЛАКА СЛОВ
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
#ПОИСК ДОКУМЕНТОВ ПО КЛЮЧЕВЫМ СЛОВАМ
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["healthcare","cure"], num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Далее идет создание упражнений

In [ ]:
#функция для создания упражнения 
def exercise(text):
    clean_text = re.sub(r'[^\w\s]', '', text) #убираем пунктуацию
    text_tokenized = word_tokenize(clean_text) #токенизируем
    text_tagged = nltk.pos_tag(text_tokenized) #тэгируем, но теперь надо создать список слов с тэгами
    list_of_tagged = [] #создаем список всех слов с "приклеенными" тэгами,который можно перебирать
    for elem in text_tagged:
        word_tagged = '_'.join(elem)
        list_of_tagged.append(word_tagged)
    tagged_string = ' '.join(list_of_tagged) #объединяем в строку, чтобы можно было пройтись регуляркой
    
    verbs_past_participle = re.findall('[a-zA-Z]+_VBN', tagged_string)
    verbs_gerund = re.findall('[a-zA-Z]+_VBG', tagged_string)
    verbs_past_tense = re.findall('[a-zA-Z]+_VBD', tagged_string)
    verbs_present_3rdperson = re.findall('[a-zA-Z]+_VBZ', tagged_string)
    all_verbs = verbs_past_participle + verbs_gerund + verbs_past_tense + verbs_present_3rdperson #список глаголов из текста (кроме базовых форм)
    verbs_string = ' '.join(all_verbs)
    verbs_clean = re.sub(r'_[a-zA-Z]*', '', verbs_string)
    
    verb_lemmas = [] #создаем список лемматизированных глаголов 
    doc = nlp(verbs_clean)
    for token in doc:
        verb_lemma = token.lemma_
        verb_lemmas.append(verb_lemma)
        
    new_text = text
    for verb in verb_lemmas: #собственно цикл, убирающий из оригинального текста предлоги и заменяющий их на пробелы
       reg_exp = fr"\s{verb}\s"
       new_text = re.sub(reg_exp, " ___ ", new_text)
    
    str = "Fill in the gaps with the verbs in the correct form:"

    return(str, verb_lemmas, new_text)

In [ ]:
#нужно загрузить наши датафреймы и взять из них только элементы из колонки documents 
education_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/education.csv")
education = education_df["documents"].tolist()

fashion_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/fashion.csv")
fashion = fashion_df["documents"].tolist()

food_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/food.csv")
food = food_df["documents"].tolist()

jobs_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/jobs.csv")
jobs = jobs_df["documents"].tolist()

health_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/health.csv")
health = health_df["documents"].tolist()

sport_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/sport.csv")
sport = sport_df["documents"].tolist()

films_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/films.csv")
films = films_df["documents"].tolist()

environment_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/environment.csv")
environment = environment_df["documents"].tolist()

technology_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/technology.csv")
technology = technology_df["documents"].tolist()

tourism_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/tourism.csv")
tourism = tourism_df["documents"].tolist()

culture_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/culture.csv")
culture = culture_df["documents"].tolist()

music_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/music.csv")
music = music_df["documents"].tolist()

politics_df = pd.read_csv(r"https://raw.githubusercontent.com/BondarenkoAnnette/HSE_CL_Project_2024/main/data/politics.csv")
politics = politics_df["documents"].tolist()

In [ ]:
topics = [politics, music, films, sport, culture, technology, environment, health, jobs, food, fashion, education]

strings = ["politics", "music", "films", "sport", "culture", "technology", "environment", "health", "jobs", "food", "fashion", "education"]

In [ ]:
print("Choose one of the following topics for your exercise:")
print("politics, music, films, sport, culture, technology, environment, health, jobs, food, fashion, education")

selected_topic = input() 

def checking(selected_topic):
    for i in range(0,12):
        if selected_topic == strings[i]:
            topic_list = topics[i]
            random_article = random.choice(topic_list)
            ex = exercise(random_article)
            answer = input("Do you want to see the article?")
            if answer == "yes":
                return ex, random_article
            else:
                return ex

print(checking(selected_topic))